In [1]:
import numpy as np

In [110]:
EPS = 1e-10

class LPSolver:
    def __init__(self, A, b, c):
        self.m = len(b)
        self.n = len(c)
        self.N = list(range(self.n)) + [-1]
        self.B = list(range(self.n, self.n + self.m))
        self.D = A
        for i in range(self.m):
            self.D[i] += [-1, b[i]]
        self.D += [(-np.array(c)).tolist() + [0, 0]]
        self.D += [[0] * (self.n) + [1, 0]]
        self.D = np.array(self.D, dtype=np.float64)
        self.N = np.array(self.N, dtype=np.int32)
        self.B = np.array(self.B, dtype=np.int32)
        
    def pivot(self, r, s):
        inv = 1 / self.D[r][s]
        for i in range(self.m + 2):
            if i != r and abs(self.D[i][s]) > EPS:
                inv2 = self.D[i][s] * inv
                self.D[i] -= self.D[r] * inv2
                self.D[i][s] = self.D[r][s] * inv2
        self.D[r, :] *= inv
        self.D[:, s] *= -inv
        self.D[r][s] = inv
        self.B[r], self.N[s] = self.N[s], self.B[r]
        
    def simplex(self, phase):
        x = self.m + phase - 1
        while True:
            s = min((j for j in range(self.n + 1) if self.N[j] != -phase),
                    key = lambda j: (self.D[x][j], self.N[j]))
            if self.D[x][s] >= -EPS:
                return True
            ris = [i for i in range(self.m) if self.D[i][s] >= EPS]
            if len(ris) == 0:
                return False
            r = min(ris, key = lambda i: (self.D[i][self.n + 1] / self.D[i][s], self.B[i]))
            self.pivot(r, s)
                    
    def solve(self):
        r = min(range(self.m), key = lambda i: self.D[i][self.n + 1])
        if self.D[r][self.n + 1] < -EPS:
            self.pivot(r, self.n)
            if not self.simplex(2) or self.D[self.m + 1][self.n + 1] < -EPS:
                return -np.inf
            for i in range(self.m):
                if self.B[i] == -1:
                    s = min(range(self.n + 1),
                           key = lambda j: (self.D[i][j], self.N[j]))
                    self.pivot(i, s)
        ok = self.simplex(1)
        x = np.zeros(self.n)
        if ok:
            for i in range(self.m):
                if self.B[i] < self.n:
                    x[self.B[i]] = self.D[i][self.n + 1]
        return (self.D[self.m][self.n + 1], x) if ok else np.inf

In [111]:
solver = LPSolver([[2, -1, 1, 0],
                   [-2, 1, -1, 0],
                   [-1, 2, 0, 1],
                   [1, -2, 0, -1]],
                  [1, -1, 1, -1],
                  [-1, -1, 2, 3])
solver.solve()

(5.0, array([0., 0., 1., 1.]))

In [112]:
# TEST 1
solver = LPSolver([[3, 1, -1, 1],
                   [-3, -1, 1, -1],
                   [5, 1, 1, -1],
                   [-5, -1, -1, 1]],
                  [4, -4, 4, -4],
                  [6, 1, 4, -5])
solver.solve()

(4.0, array([0., 4., 0., 0.]))

In [120]:
# TEST 2
solver = LPSolver([[1, -3, -1, -2],
                   [-1, 3, 1, 2],
                   [1, -1, 1, 0],
                   [-1, 1, -1, 0]],
                  [-4, 4, 0, 0],
                  [1, 2, 3, -1])
solver.solve()

(6.0, array([2., 2., 0., 0.]))

In [114]:
# TEST 3
solver = LPSolver([[1, 1, 0, 2, 1],
                   [-1, -1, 0, -2, -1],
                   [1, 1, 1, 3, 2],
                   [-1, -1, -1, -3, -2],
                   [0, 1, 1, 2, 1],
                   [0, -1, -1, -2, -1]],
                  [5, -5, 9, -9, 6, -6],
                  [1, 2, 1, -3, 1])
solver.solve()

(10.999999999999998, array([3., 2., 4., 0., 0.]))

In [115]:
# TEST 4
solver = LPSolver([[1, 1, 2, 0, 0],
                   [-1, -1, -2, 0, 0],
                   [0, -2, -2, 1, -1],
                   [0, 2, 2, -1, 1],
                   [1, -1, 6, 1, 1],
                   [-1, 1, -6, -1, -1]],
                  [4, -4, -6, 6, 12, -12],
                  [1, 1, 1, -1, 1])
solver.solve()

(10.0, array([4., 0., 0., 1., 7.]))

In [116]:
# TEST 5
solver = LPSolver([[1, 1, -1, -10],
                   [-1, -1, 1, 10],
                   [1, 14, 10, -10],
                   [-1, -14, -10, 10]],
                  [0, 0, 11, -11],
                  [1, -4, 3, -10])
solver.solve()

(4.0, array([1., 0., 1., 0.]))

In [117]:
# TEST 6
solver = LPSolver([[1, 3, 3, 1],
                   [2, 0, 3, -1]],
                  [3, 4],
                  [1, -5, -1, 1])
solver.solve()

(3.0, array([2.33333333, 0.        , 0.        , 0.66666667]))

In [118]:
# TEST 7
solver = LPSolver([[3, 1, 1, 1, -2],
                   [-3, -1, -1, -1, 2],
                   [6, 1, 2, 3, -4],
                   [-6, -1, -2, -3, 4],
                   [10, 1, 3, 6, -7],
                   [-10, -1, -3, -6, 7]],
                  [10, -10, 20, -20, 30, -30],
                  [1, 1, -1, 1, -2])
solver.solve()

(-10.0, array([ 0.,  0., 10.,  0.,  0.]))